# **Loading Necessary Lib**

In [ ]:
!pip install boltiotai
!pip install --upgrade boltiotai
import pandas as pd
import re
from boltiotai import openai

# **Importing Dataset**

In [ ]:
# Load the cleaned dataset
df = pd.read_csv("/content/World Important Dates.csv")
df

,Sl. No,Name of Incident,Date,Month,Year,Country,Type of Event,Place Name,Impact,Affected Population,Important Person/Group Responsible,Outcome
0,1,Indus Valley Civilization Flourishes,Unknown,Unknown,2600 BC,India,Civilization,Indus Valley,Development of one of the world's earliest urb...,Local inhabitants,Indus Valley people,Positive
1,2,Battle of the Ten Kings,Unknown,Unknown,1400 BC,India,Battle,Punjab,Rigvedic tribes consolidated their control ove...,Rigvedic tribes,Sudas,Positive
2,6,Establishment of the Delhi Sultanate,Unknown,Unknown,1206,India,Political,Delhi,Muslim rule established in parts of India,People of Delhi and surrounding regions,QutbUnknownudUnknowndin Aibak,Mixed
3,7,Battle of Panipat,21,April,1526,India,Battle,Panipat,Foundation of the Mughal Empire in India,Northern Indian kingdoms,Babur,Mixed
4,8,Establishment of British Raj,1,May,1858,India,Colonial,Whole India,Start of direct British governance in India,Indian subcontinent,British East India Company/Empire,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...
1091,1147,First Mexican Empire Declared,28,September,1821,Mexico,Political,Mexico,Brief establishment of an empire soon transiti...,Mexicans,Agustín de Iturbide,Positive
1092,1148,U.S.UnknownMexican War,25,April,1846,Mexico,Military,Northern Mexico,Loss of vast territories to the United States,Mexicans,US,Negative
1093,1149,Reform Wars,Unknown,Unknown,1857,Mexico,Civil War,Mexico,Liberal vs. Conservative conflict leading to c...,Mexicans,Benito Juárez,Mixed
1094,1150,French Intervention in Mexico,Unknown,Unknown,1862,Mexico,Military Intervention,Mexico,Establishment and fall of the Second Mexican E...,Mexicans,Napoleon III,Negative


In [ ]:
df.isnull().sum() #Checking if any null exists in the dataset

,0
Name of Incident,0
Date,0
Month,0
Year,0
Country,0
Type of Event,0
Place Name,0
Impact,0
Affected Population,0
Important Person/Group Responsible,0


# **Pre-Processing Data**

In [ ]:
df = df.drop(columns=['Sl. No'], errors='ignore') #droping unnecessary column

In [ ]:
def clean_name(name):
    if pd.isna(name) or "Unknown" in name:
        return "Unknown (No specific individual)"
    return re.sub(r'Unknown+', 'Unknown', name)  # Replaced repeated 'Unknown' parts

# Applying cleaning function
df['Important Person/Group Responsible'] = df['Important Person/Group Responsible'].apply(clean_name)


In [ ]:
df

,Name of Incident,Date,Month,Year,Country,Type of Event,Place Name,Impact,Affected Population,Important Person/Group Responsible,Outcome
0,Indus Valley Civilization Flourishes,Unknown,Unknown,2600 BC,India,Civilization,Indus Valley,Development of one of the world's earliest urb...,Local inhabitants,Indus Valley people,Positive
1,Battle of the Ten Kings,Unknown,Unknown,1400 BC,India,Battle,Punjab,Rigvedic tribes consolidated their control ove...,Rigvedic tribes,Sudas,Positive
2,Establishment of the Delhi Sultanate,Unknown,Unknown,1206,India,Political,Delhi,Muslim rule established in parts of India,People of Delhi and surrounding regions,Unknown (No specific individual),Mixed
3,Battle of Panipat,21,April,1526,India,Battle,Panipat,Foundation of the Mughal Empire in India,Northern Indian kingdoms,Babur,Mixed
4,Establishment of British Raj,1,May,1858,India,Colonial,Whole India,Start of direct British governance in India,Indian subcontinent,British East India Company/Empire,Negative
...,...,...,...,...,...,...,...,...,...,...,...
1091,First Mexican Empire Declared,28,September,1821,Mexico,Political,Mexico,Brief establishment of an empire soon transiti...,Mexicans,Agustín de Iturbide,Positive
1092,U.S.UnknownMexican War,25,April,1846,Mexico,Military,Northern Mexico,Loss of vast territories to the United States,Mexicans,US,Negative
1093,Reform Wars,Unknown,Unknown,1857,Mexico,Civil War,Mexico,Liberal vs. Conservative conflict leading to c...,Mexicans,Benito Juárez,Mixed
1094,French Intervention in Mexico,Unknown,Unknown,1862,Mexico,Military Intervention,Mexico,Establishment and fall of the Second Mexican E...,Mexicans,Napoleon III,Negative


# **ChatBot**

In [ ]:
class HistoricalChatbot:
    def __init__(self, api_key):
        """
        Initializes the chatbot with the Bolt IoT API key.
        """
        openai.api_key = api_key
        self.messages = [{"role": "system", "content": "You are a historical timeline expert. Provide users with detailed responses about historical events."}]

    def extract_keywords(self, query):
        """
        Extracts key terms from the query by removing common words.
        """
        stop_words = {"when", "happened", "?", "did", "happen", "was", "the", "of", "about", "tell", "me", "who", "is", "What","Taken"}
        words = re.sub(r'[^a-zA-Z0-9 ]', '', query.lower()).split()
        keywords = [word for word in words if word not in stop_words]
        return " ".join(keywords)

    def get_event(self, query):
        """
        Finds the most relevant historical event based on user query.
        """
        keywords = self.extract_keywords(query)
        relevant_events = df[df.apply(lambda row: all(word in row.to_string().lower() for word in keywords.split()), axis=1)]

        if relevant_events.empty:
            return None  # Return None if no matching event is found

        event = relevant_events.iloc[0]  # To get the most relevant result
        return (f"Incident: {event['Name of Incident']}\n"
                f"Date: {event['Date']} {event['Month']} {event['Year']}\n"
                f"Country: {event['Country']}\n"
                f"Place: {event['Place Name']}\n"
                f"Impact: {event['Impact']}\n"
                f"Important Person/Group Responsible: {event['Important Person/Group Responsible']}\n")

    def chat(self):
        """
        Handles the chatbot conversation.
        """
        print("To end conversation, type END")
        while True:
            query = input("You: ")
            if query.upper() == "END":
                break

            event_info = self.get_event(query)

            if event_info:
                content = event_info
            else:
                print("Searching Web for the information...")
                self.messages.append({'role': 'user', 'content': query})
                response = openai.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages
                )

                # Ensure we properly extract the response content
                if isinstance(response, dict) and "choices" in response and response["choices"]:
                    content = response["choices"][0]["message"]["content"].strip()
                else:
                    content = "I'm sorry, but I couldn't retrieve the information."

                self.messages.append({'role': 'assistant', 'content': content})

            print("\nChatbot:", content)

# Initialize chatbot with Bolt IoT API Key
bot = HistoricalChatbot(api_key="YM3r4-acRVnoFzYYEHnI9uj8C98qan7fMunvPn_q8jA")
bot.chat()

To end conversation, type END
You: Tell me about Asiatic Lion Census Showing Population Increase?

Chatbot: Incident: Asiatic Lion Census Showing Population Increase
Date: Unknown May 2020
Country: India
Place: Gujarat
Impact: Indicated successful conservation efforts, with an increase in the Asiatic lion population
Important Person/Group Responsible: Wildlife Conservationists, Government of Gujarat

You: What is the current population of Asiatic lion in India as of feb-2025?
Searching Web for the information...

Chatbot: I'm sorry, but as of February 2025, I do not have real-time data on the exact population of Asiatic lions in India. However, as of the last official census conducted in June 2020, the population of Asiatic lions in the Gir Forest National Park and surrounding areas in the state of Gujarat was estimated to be around 674 individuals. It is important to note that population figures for wildlife species are subject to change due to various factors such as habitat loss, co